In [72]:
import numpy as np

from strat import RSIRisingFalling

from BinanceExchangeSettings import BinanceTokenSettings

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest

np.set_printoptions(formatter={"float_kind":"{:0.2f}".format})

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
candles = dl_ex_candles(
    exchange="binance_us",
    symbol='AVAXUSDT',
    #symbol='PEPEUSDT',
    timeframe='30m',
    candles_to_dl=1344
    #since_datetime=datetime(2024,3,3),
    #until_datetime=datetime(2024,3,7),   
)

In [74]:
short_start = RSIRisingFalling(
    long_short="short",
    rsi_length=np.array([14]),
    rsi_is_above=np.arange(50,76,5),
)

short_start.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
short_start.plot_signals(candles=candles)


In [75]:
long_start = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(35,61,5),
)

long_start.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
long_start.plot_signals(candles=candles)

In [76]:
backtest_settings = BacktestSettings()

# exchange_settings=ExchangeSettings(
#     asset_tick_step=6,
#     leverage_mode=1,
#     leverage_tick_step=2,
#     limit_fee_pct=0.0003,
#     market_fee_pct=0.0006,
#     max_asset_size=100.0,
#     max_leverage=50.0,
#     min_asset_size=0.001,
#     min_leverage=1.0,
#     mmr_pct=0.004,
#     position_mode=3,
#     price_tick_step=1,    
# )


# exchange_settings=ExchangeSettings(
#       asset_tick_step=3,
#       leverage_mode=1,
#       leverage_tick_step=2,
#       limit_fee_pct=0.0003,
#       market_fee_pct=0.0006,    
      
#       max_asset_size=100.0,
#       max_leverage=150.0,
#       min_asset_size=0.001,
#       min_leverage=1.0,
#       mmr_pct=0.004,
#       position_mode=3,
#       price_tick_step=1,    
#   )


exchange_settings=ExchangeSettings(
    asset_tick_step= 6,
    leverage_mode= 1,
    leverage_tick_step= 2,
    limit_fee_pct= 0.0003,
    market_fee_pct= 0.0006,
    max_asset_size= 2000.0,
    max_leverage= 150.0,
    min_asset_size= 0.000157,
    min_leverage= 1.0,
    mmr_pct= 0.004,
    position_mode= 3,
    price_tick_step= 0
)

#exchange_settings = BinanceTokenSettings('BTCUSDT')
#print(exchange_settings.asset_tick_step)  # Example usage

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1,0.25, 0.5]),
    sl_based_on_lookback=np.array([20,50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5,1.0]),
    trail_sl_when_pct=np.array([1,2]),
)

TypeError: BinanceTokenSettings.__init__() got an unexpected keyword argument 'asset_tick_step'

In [ ]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_start
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 48
Total combinations of settings to test: 288
Total candles: 1,344
Total candles to test: 387,072


In [ ]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(10)


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
0,0,0,7.0,7,0,23.055,100.000,0.957,16.893,1230.546,230.546
4,0,4,7.0,7,0,23.055,100.000,0.957,16.893,1230.546,230.546
12,0,12,7.0,7,0,19.143,100.000,0.950,15.386,1191.433,191.433
8,0,8,7.0,7,0,19.143,100.000,0.950,15.386,1191.433,191.433
2,0,2,7.0,6,1,16.197,85.714,0.906,16.490,1161.966,161.966
6,0,6,7.0,6,1,16.197,85.714,0.906,16.490,1161.966,161.966
28,0,28,7.0,7,0,31.331,100.000,0.806,17.475,1313.309,313.309
24,0,24,7.0,7,0,31.331,100.000,0.806,17.475,1313.309,313.309
36,0,36,7.0,7,0,28.968,100.000,0.786,15.415,1289.676,289.676
32,0,32,7.0,7,0,28.968,100.000,0.786,15.415,1289.676,289.676


In [ ]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_start,
    dos_index=24,
    ind_set_index=0,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 5.0,
    sl_based_on_add_pct = 0.001,
    sl_based_on_lookback = 20,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.005,
    trail_sl_when_pct = 0.01,
)
IndicatorSettingArrays(
    rsi_is_above = nan,
    rsi_is_below = 35.0,
    rsi_length = 14,
)


d:\Business\Crypto\bot\QuantFredom\env\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

